## Coleta e limpeza de Dados Semi-estruturados.

In [ ]:
import pandas as pd # Importa a biblioteca Pandas, usada para manipulação e análise de dados.
import json # Importa a biblioteca json, usada para manipulação de dados JSON.
import xml.etree.ElementTree as ET # Importa a biblioteca xml.etree.ElementTree, usada para manipulação de dados XML.
import requests # Importa a biblioteca requests, usada para realizar requisições HTTP.

In [ ]:
api_url = 'https://jsonplaceholder.typicode.com/users' # URL da API de citações.
response = requests.get(api_url) # Fazendo a requisição HTTP.
data = response.json() # Converte a resposta JSON para um dicionário.
df_json = pd.DataFrame(data) # Criando um DataFrame a partir do dicionário.

df_json_normal = pd.json_normalize(data) # Normalizando o DataFrame para facilitar a visualização.

df_json
df_json_normal
df_json_normal.info() # Exibindo informações sobre o DataFrame.

In [ ]:
nested = {
    'nome': 'x',
    'email': 'lindo@gmail.com',
    'addrress': {
        'rua': 'Rua de Casa',
        'cidade': 'Santo André',
        'estado': 'UF',
        'cep': '12345678'
    } 
}

df = pd.json_normalize(nested)
df

In [ ]:
df_json['company'] = None # Criando uma nova coluna com dados de empresa.
df_fil = df_json.fillna('Não Preenchido', inplace=True) # Preenchendo colunas com dados ausentes com um valor específico.
df_fil

In [ ]:
df_json['company'] = None # Criando uma nova coluna com dados de empresa.
df_drop = df_json.dropna(axis=1) # Removendo colunas com dados ausentes.
df_drop

In [ ]:
df_json_exploded = df_json.explode('address') # Explodindo a coluna 'address' em uma nova coluna para cada item do endereço.
df_json_exploded

In [ ]:
xml_data = """
<data>
    <user>
        <name>joao</name>
        <email>joao@gmail.com</email>
        <cidade>santo andre</cidade>
    </user>
    <user>
        <name>maria</name>
        <email>maria@gmail.com</email>
        <cidade>sao paulo</cidade>
    </user>
</data>
"""

root = ET.fromstring(xml_data) # Criando um objeto ElementTree a partir do XML.	
users = [] # Lista para armazenar os dados dos usuários.	

for user in root.findall("user"): # Iterando sobre cada elemento 'user' do XML.	
    users.append({ # Criando um dicionário com os dados do usuário.	
        'name':user.find('name').text if user.find('name') is not None else None,
        'email':user.find('email').text if user.find('email') is not None else None,
        'cidade':user.find('cidade').text if user.find('cidade') is not None else None

    })

df_xml = pd.DataFrame(users) # Criando um DataFrame a partir dos dados dos usuários.	
df_xml

In [ ]:
df_dups = pd.DataFrame({'id': [1, 1, 3, 4], 'nome': ['Alan', 'Alan', 'Brenda', 'Aurora Flor']}) # DataFrame com dados duplicados.
print('Antes') 
print(df_dups)

df_unico = df_dups.drop_duplicates() # Removendo dados duplicados.

print('Depois')

print(df_unico)

In [ ]:
df_padrao = pd.DataFrame({'datas': ['2022-01-01', 'feb 24, 2024', '2022-01-03', '12-01-2024']}) # DataFrame com dados incorretos de data.

df_padrao['datas'] = pd.to_datetime(df_padrao['datas'], format='mixed') # Convertendo dados de data para o formato padrão.

print(df_padrao) #  Exibindo o DataFrame com dados corrigidos.

In [ ]:
with open('data.json', 'r', encoding='utf-8') as f: 
    data_arq = json.load(f)

data_arq[10]

In [ ]:
print('chaves', data_arq[0].keys()) # Exibindo as chaves do primeiro elemento do JSON.

In [ ]:
df_yelp = pd.json_normalize(data_arq) # Normalizando o DataFrame para facilitar a visualização.
df_yelp.head()

In [ ]:
df_yelpcustom = pd.DataFrame([{
    'business_id': d.get('business_id'),
    'name': d.get('name'),
    'city': d.get('city'),
    'review_count': d.get('review_count'),
    'stars': d.get('stars'),
    'categories': d.get('categories')
} for d in data_arq])
print(df_yelpcustom.info())

In [ ]:
df_yelpcustom.fillna('Desconhecido', inplace=True) # Preenchendo colunas com dados ausentes com um valor específico.
print(df_yelpcustom.info()) # Exibindo informações sobre o DataFrame.

In [ ]:
df_yelpatri = pd.DataFrame([{
     'business_id': d.get('business_id'), 
    'Wifi': (d.get('attributes') or {}).get('WiFi', 'Unknown'), # Busca o valor da chave 'WiFi' no dicionário 'attributes' ou usa 'Unknown' caso não seja encontrado.
    'OutdoorSeating': (d.get('attributes') or {}).get('OutdoorSeating', 'Unknown'), # Busca o valor da chave 'OutdoorSeating' no dicionário 'attributes' ou usa 'Unknown' caso não seja encontrado.
} for d in data_arq])

print(df_yelpatri)

In [ ]:
data_arq[0].keys()

In [ ]:
weekdays = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'] # Dias da semana.

df_horario = pd.DataFrame([{
    'business_id': d.get('business_id'),
    # Busca o horário para cada dia da semana no dicionário 'hours' ou usa 'Fechado' caso não seja encontrado.
    **{day: (d.get('hours') or {} ).get(day, 'Fechado') for day in weekdays} 
} for d in data_arq])

df_horario

In [ ]:
# Fazendo um merge left join entre os DataFrames df_yelpcustom e df_yelpatri.
df_merge = df_yelpcustom.merge(df_yelpatri, on = 'business_id', how = 'left') 

# Fazendo um merge left join entre o resultado anterior e df_horario.
df_merge = df_merge.merge(df_horario, on = 'business_id', how = 'left') 

df_merge.info()


In [ ]:
df_merge_explode = df_merge.assign(categories = df_merge['categories'].str.split(', ')) # Explodindo a coluna 'categories' em uma nova coluna para cada item da lista.
df_merge_explode = df_merge_explode.explode('categories') # Reindexando o DataFrame para eliminar as linhas duplicadas.
df_merge_explode.head(10) # Exibindo as primeiras 10 linhas do DataFrame.
len(df_merge_explode) # Exibindo o número de restaurantes no DataFrame.

df_restaurantes = df_merge_explode[df_merge_explode['categories'] == 'Restaurants'] # Filtrando os restaurantes.
len(df_restaurantes) # Exibindo o número de restaurantes no DataFrame.